# Imports

In [102]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os 
import glob
import sys
from datetime import datetime, timedelta
%run read_matlab_file.ipynb

read_matlab_file.ipynb loaded


# TODO
-Make sure the right coefs are used. NOT CORRECT. Solution: sorted the list

-Make sure erg is calibrated properly

-Make a dataframe containing the same information as AL have produced

In [7]:
path_coef = "/Users/andreas/phys/master/GRB/ASIM/COEFS/BGO/" #MAC
#path_coef = "/Home/siv30/wad005/master/GRB/ASIM/COEFS/BGO/" #UiB

In [78]:
coef_filenames = glob.glob(path_coef + "*.txt")
coef_filenames.sort()

In [79]:
dfs_list = []
for i in np.arange(0,len(coef_filenames)):
    df = pd.read_csv(coef_filenames[i],delim_whitespace=True)
    #joining date and time 
    dt_list = []
    date_arr = df["DATE"].values
    time_arr = df["TIME"].values
    for i in np.arange(0,len(date_arr)):
        dt_str = date_arr[i]+ " " + time_arr[i]
        dt = datetime.strptime(dt_str, "%d-%b-%Y %H:%M:%S")
        dt_list.append(dt)
    df["datetime"] = dt_list
    df = df.drop(columns=["DATE","TIME"])
    dfs_list.append(df)

In [80]:
dfs_coefs = {"dau_1_det_1":dfs_list[0],"dau_1_det_2":dfs_list[1],"dau_1_det_3":dfs_list[2],"dau_2_det_1":dfs_list[3],
      "dau_2_det_2":dfs_list[4],"dau_2_det_3":dfs_list[5],"dau_3_det_1":dfs_list[6],"dau_3_det_2":dfs_list[7],
      "dau_3_det_3":dfs_list[8],"dau_4_det_1":dfs_list[9],"dau_4_det_2":dfs_list[10],"dau_4_det_3":dfs_list[11]}

In [81]:
dau_det_list = []
for i in np.arange(1,5):
        for j in np.arange(1,4):
            dau_det_list.append("dau_{}_det_{}".format(i,j))

In [107]:
def get_coefs(T0):
    """Input only T0 for the GRB event.
    Function will return a, b, c for all 12 detectors"""
    dau_det_coefs = {}
    
    for i in np.arange(1,5): #generating the output dict containing the coefs
        for j in np.arange(1,4):
            dau_det_coefs["dau_{}_det_{}".format(i,j)] = None
            
    #Need to check that is choosing a time that is lower. 
    #Example: finding time that is closest. That time is greater than the provided (wrong). 
    #--> should therefore choose the previous index
    
    #>> Solution: use method="pad" to use the previous index if no excact max
    for dau_det in dau_det_list:
        row = dfs_coefs[dau_det].iloc[dfs_coefs[dau_det].set_index('datetime').index.get_indexer([T0], method='pad')]
        dau_det_coefs[dau_det] = {"a":row["a"].values[0], "b": row["b"].values[0], "c":row["c"].values[0]}
    return dau_det_coefs

In [165]:
def convert_erg(erg,T0):
    """should specify the T0 for the event"""
    
    """keV = (A*CHANNEL^2) + (B*CHANNEL) + C"""
    keV_list = []
    cal_abc = []
    dau_det_coefs = get_coefs(T0)
    
    for i in np.arange(0,len(erg)):
        dau_val = dau[i]
        det_val = det[i]
        erg_val = erg[i]
        

        a = dau_det_coefs["dau_{}_det_{}".format(dau_val,det_val)]["a"]
        b = dau_det_coefs["dau_{}_det_{}".format(dau_val,det_val)]["b"]
        c = dau_det_coefs["dau_{}_det_{}".format(dau_val,det_val)]["c"]
        cal_abc.append([a,b,c])
        
        keV = a*erg_val**2 + b*erg_val + c
        keV_list.append(keV)
        
    return np.array(keV_list), np.array(cal_abc)

In [186]:
#tus, erg, typ, adr, dau, det, date, T0 = read_matlab_file_with_BGO_data(trig_dict_HED["GRB200415"][0])

In [185]:
#keV,cal_abc = convert_erg(erg,T0)

In [174]:
def create_df(tus,erg,typ,adr,dau,det,keV,cal_abc):

    for arr in [tus,erg,dau,det,a,b,c,keV]:
        if len(arr) != len(tus):
            return print("length of arrays is not equal")
        else:
            continue
    df = pd.DataFrame({"tus":tus,"erg":erg,"typ":typ,"adr":adr,"dau":dau,"det":det,"keV":keV, "a":cal_abc[:,0], "b":cal_abc[:,1],"c":cal_abc[:,2]})
    cols = ['a', 'b','c']
    df['cal_abc'] = df[cols].apply(lambda row: ', '.join(row.values.astype(str)), axis=1)
    df = df.drop(columns=cols)
    return df

In [183]:
#df = create_df(tus,erg,typ,adr,dau,det,keV,cal_abc)
#df.head()

In [184]:
#df = df.sort_values(by=['adr','tus'])
#df.head()

In [179]:
def export_df(df,path=None):
    df = df.sort_values(by=['adr','tus'])
    df.to_csv(path_or_buf=path,index=False)
    return None

In [182]:
#export_df(df,path="/Users/andreas/phys/master/GRB/calibration/GRB_150420_better.csv")

In [16]:
print("convert_erg.ipynb loaded")

convert_erg.ipynb loaded


In [180]:
df

,tus,erg,typ,adr,dau,det,keV,cal_abc
2,-9.703999e+05,54,0,1,1,1,534.272300,"0.005245, 8.4154, 64.54628"
3,-9.699581e+05,216,0,1,1,1,2126.983400,"0.005245, 8.4154, 64.54628"
4,-9.696501e+05,197,0,1,1,1,1925.933285,"0.005245, 8.4154, 64.54628"
5,-9.694163e+05,32,0,1,1,1,339.209960,"0.005245, 8.4154, 64.54628"
14,-9.679059e+05,1727,0,1,1,1,30241.306685,"0.005245, 8.4154, 64.54628"
...,...,...,...,...,...,...,...,...
14389,1.019510e+06,40,0,12,4,3,534.008900,"0.002252, 12.9039, 14.2497"
14391,1.020160e+06,127,0,12,4,3,1689.367508,"0.002252, 12.9039, 14.2497"
14429,1.025592e+06,49,0,12,4,3,651.947852,"0.002252, 12.9039, 14.2497"
14435,1.026366e+06,1833,0,12,4,3,31233.568428,"0.002252, 12.9039, 14.2497"
